In [1]:
import json
import os
import requests
import sys

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from wbe_odm import odm
from wbe_odm.odm_mappers import mcgill_mapper
from wbe_odm.odm_mappers import excel_template_mapper

In [2]:
def get_available_sites(odm_data):
    samples = odm_data.sample
    available_sites = list(samples["siteID"].unique())
    site_dico = {}
    for i, site in enumerate(available_sites):
        if site == "":
            continue
        site_dico[i] = site
    return site_dico

### Quebec



In [4]:
path_to_parser_help = "../Data/Lab/McGill/Final/"
lab_data = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Quebec_final.xlsx"
static_data = "/Users/jeandavidt/Desktop/latest-data/Ville de Quebec - All data - v1.1.xlsx"
mapping = path_to_parser_help + "mcgill_map.csv"
sheet_name = "QC Data Daily Samples (McGill)"
lab_id = "frigon_lab"
start_date = "2021-01-01"
end_date = None
qc_lab = mcgill_mapper.McGillMapper()
qc_lab.read(lab_data, static_data, sheet_name, lab_id)
qc_odm = odm.Odm()
qc_odm.load_from(qc_lab)

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [12]:
ww = qc_lab.ww_measure
ww.loc[ww["type"].isin(["covN2", "nPMMoV"]) & ww["unit"].isin(["gc/ml"]), ["sampleID", "type", "value", "unit"]]

variableName,sampleID,type,value,unit
4103,frigon_lab_spb_1_qc_01_cptp24h _rawww_1,nPMMoV,242244.413795,gc/ml
4104,frigon_lab_spb_1_qc_02_cptp24h _rawww_1,nPMMoV,88623.624672,gc/ml
4105,frigon_lab_spb_1_qc_01_cptp24h _rawww_1,nPMMoV,83259.516472,gc/ml
4106,frigon_lab_spb_1_qc_02_cptp24h _rawww_1,nPMMoV,115650.548175,gc/ml
4107,frigon_lab_spb_1_qc_01_cptp24h _rawww_1,nPMMoV,303896.921506,gc/ml
...,...,...,...,...
5958,qc_08_ps24h_filter_2021-04-21_1,covN2,0.011484,gc/ml
5959,qc_09_ps24h_filter_2021-04-21_1,covN2,0.008831,gc/ml
5960,frigon_lab__reference_1,covN2,0.062857,gc/ml
5961,frigon_lab__negative_1,covN2,0.492316,gc/ml


In [5]:
qc_lab_data.sample["sampleID"].head()

0    qc_01_cptp24h_pstgrit_2021-01-15_1
1    qc_02_cptp24h_pstgrit_2021-01-15_1
2    qc_01_cptp24h_pstgrit_2021-01-16_1
3    qc_02_cptp24h_pstgrit_2021-01-16_1
4    qc_01_cptp24h_pstgrit_2021-01-17_1
Name: sampleID, dtype: object

In [5]:
bad_samples = [
    "qc_01_cptp24h_pstgrit_2021-02-12_1",
    "qc_01_cptp24h_pstgrit_2021-02-13_1",
    "qc_01_cptp24h_pstgrit_2021-02-14_1",
    "qc_01_cptp24h_pstgrit_2021-02-19_1",
    "qc_01_cptp24h_pstgrit_2021-02-20_1",
    "qc_01_cptp24h_pstgrit_2021-02-21_1",
    "qc_01_cptp24h_pstgrit_2021-02-22_1",
    "qc_01_cptp24h_pstgrit_2021-02-23_1",
    "qc_01_cptp24h_pstgrit_2021-03-01_1",
    "qc_01_cptp24h_pstgrit_2021-03-23_1",
    
    "qc_02_cptp24h_pstgrit_2021-02-12_1",
    "qc_02_cptp24h_pstgrit_2021-02-13_1",
    "qc_02_cptp24h_pstgrit_2021-02-14_1",
    "qc_02_cptp24h_pstgrit_2021-02-15_1",
    "qc_02_cptp24h_pstgrit_2021-02-16_1",
    "qc_02_cptp24h_pstgrit_2021-02-19_1",
    "qc_02_cptp24h_pstgrit_2021-02-20_1",
    "qc_02_cptp24h_pstgrit_2021-02-21_1",
    "qc_02_cptp24h_pstgrit_2021-02-22_1",
    "qc_02_cptp24h_pstgrit_2021-02-23_1",
    "qc_02_cptp24h_pstgrit_2021-03-23_1",
    "qc_02_cptp24h_pstgrit_2021-03-24_1",
]
qc_lab_data = apply_quality_flag(qc_lab_data, bad_samples, ["covN2", "nPMMoV"])

In [11]:
def sample_time(row):
    comp_start = row["dateTimeStart"]
    grab = row["dateTime"]
    if pd.isna(comp_start):
        return grab
    else:
        return comp_start
def date_from_sample_id(sample_id, samples):
    sample_slice = samples.loc[samples["sampleID"] == sample_id]
    try:
        return sample_slice.iloc[0]["reportDate"]
    except IndexError:
        return pd.NaT

In [15]:
def get_normalized_data(odm_data, site_id):
    samples = odm_data.sample
    samples["reportDate"] = samples.apply(sample_time, axis=1)
    samples_from_site = samples.loc[samples["siteID"].str.lower() == site_id, "sampleID"].unique()
    
    measures = odm_data.ww_measure
    sample_filt = (measures["sampleID"].isin(samples_from_site))
    viral_filt = (measures["type"].isin(["covN2", "nPMMoV"]))
    unit_filt = (measures["unit"] == "gc/ml")
    quality_filt = (measures["qualityFlag"] == False)
    measures = measures.loc[sample_filt & viral_filt & unit_filt & quality_filt, ["sampleID", "type", "value"]]
    samples = odm_data.sample
    measures["sampleDate"] = measures["sampleID"].apply(
        lambda row: date_from_sample_id(row, samples))
    measures = measures.drop_duplicates(subset=["sampleID", "type"])
    measures = measures.pivot(index=["sampleID", "sampleDate"], columns="type", values="value")
    if "covN2" not in measures.columns:
        print(f"no sars measurements found for site {site_id}")
        return None
    measures["normalized"] = measures["covN2"] / measures["nPMMoV"]
    measures = measures.sort_values("sampleDate")
    measures = measures.reset_index()
    return measures.dropna(subset=["normalized"])


def prettify_name(name):
    name_lst = name.split(" ")
    name_lst = [x.title() for x in name_lst]
    name = " ".join(name_lst)
    name = name.replace("Quebec", "Québec")
    name = name.replace("Montreal", "Montréal")
    return name

def graph_normalized(odm_data, region_name, site_list, max_normalized):
    fig = make_subplots(rows=1, cols=1,
                    specs=[[{"secondary_y": True}]])
    traces = []
    for site in site_list:
        name = get_site_name(odm_data, site)
        site_measures = get_normalized_data(odm_data, site)
        if site_measures is None:
            print(site)
            continue
        trace = go.Scatter(
            x=site_measures["sampleDate"],
            y=site_measures["normalized"],
            name=prettify_name(name),
            mode="lines+markers",
            text=site_measures["sampleID"],
            hoverinfo="text"
        )
        traces.append(trace)
    for trace in traces:
        fig.add_trace(trace, secondary_y=True)    
    fig.update_layout(
        xaxis_title="Date",
        xaxis_tick0="2020-12-27",
        xaxis_dtick=7 * 24 * 3600000,
        xaxis_tickformat="%d-%m-%Y",
        xaxis_tickangle=30, plot_bgcolor="white",
        xaxis_gridcolor="rgba(100,100,100,0.10)",
        yaxis_gridcolor="rgba(0,0,0,0)",
        xaxis_ticks="outside"
    )
    fig.update_yaxes(title="SARS-CoV-2 / PMMoV", secondary_y=True, range=[0, max_normalized])
    if region_name == "Capitale-Nationale":
        region_name = "Région de Québec"
    fig.update_layout(title=dict(text=f"Surveillance SRAS-CoV-2 via les eaux usées<br>{region_name}"))
    return fig

def get_cases_from_ledevoir(region_name, start_date=None, end_date=None):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    response = requests.get("https://ledevoir-coronavirus.herokuapp.com/api/v2/reports/ca/qc")
    j = json.loads(response.text)
    region_codes = {j["regions"][i]["name"]: i for i in range(len(j["regions"]))}
    region_code = region_codes[region_name]
    cases = pd.DataFrame(j["regions"][region_code]["data"])
    cases["date"] = pd.to_datetime(cases["date"])
    cases = cases.set_index("date")
    if pd.isna(start_date) and pd.isna(end_date):
        pass
    elif pd.isna(end_date):
        cases = cases.loc[start_date:]
    elif pd.isna(start_date):
        cases = cases.loc[:end_date]
    else:
        cases = cases.loc[start_date : end_date]
    return cases

def create_graph(data, region, sites, max_norm=None):
    graph = graph_normalized(data, region, sites, max_norm)
    cases = get_cases_from_ledevoir(region, start_date="2021-01-01")
    cases_trace = go.Bar(x=cases.index, y=cases["dc"], name="Nouveaux cas<br>journaliers", marker=dict(opacity=0.3))
    graph.add_trace(cases_trace, secondary_y=False)
    graph.update_layout(legend=dict(yanchor="top", xanchor="left", orientation="h", y=1.1, x=0))
    graph.update_yaxes(title="Nouveaux cas",side="right", secondary_y=False)
    graph.update_yaxes(side="left", secondary_y=True)
    graph.add_layout_image(
    dict(
        source="https://www.centreau.ulaval.ca/fileadmin/Documents/Image_de_marque/102378_MODIF_LOGO-CENTREAU_noir.jpg",
        xref="paper", yref="paper",
        x=1, y=1.00,
        sizex=0.25, sizey=0.25,
        xanchor="right", yanchor="bottom"
        )
    )
    graph.show()
    return graph

def get_site_name(odm_data, site_id):
    sites = odm_data.site
    site_rows = sites.loc[sites["siteID"].str.lower() == str(site_id).lower()]
    if len(site_rows) > 1:
        return ""
    first_row =  site_rows.iloc[0]
    return first_row.loc["name"]

region="Capitale-Nationale"
sites = ["QC_1.0", "QC_2.0"]
max_norm = 0.2
qc_graph = create_graph(qc_lab_data, region, sites, max_norm)

IndexError: single positional indexer is out-of-bounds

In [24]:
path_to_parser_help = "../Data/Lab/McGill/Final/"
lab_data = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Montreal_final.xlsx"
static_data = path_to_parser_help + "mcgill_static.xlsx"
mapping = path_to_parser_help + "mcgill_map.csv"
sheet_name = "Mtl Data Daily Samples (Poly)"
lab_id = "frigon_lab"
start_date = None
end_date = None
poly_lab_data = load_from_mcgill(lab_data, static_data, mapping, sheet_name, lab_id, start_date=start_date, end_date=end_date)

UsageError: Line magic function `%%timeit` not found.


In [22]:
path_to_parser_help = "../Data/Lab/McGill/Final/"
lab_data = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Montreal_final.xlsx"
static_data = path_to_parser_help + "mcgill_static.xlsx"
mapping = path_to_parser_help + "mcgill_map.csv"
sheet_name = "Mtl Data Daily Samples (McGill)"
lab_id = "frigon_lab"
start_date = None
end_date = None
mcgill_lab_data = load_from_mcgill(lab_data, static_data, mapping, sheet_name, lab_id, start_date=start_date, end_date=end_date)

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed



In [48]:
region="Montréal"
sites = ["mtl_01", "mtl_05", "mtl_16", "mtl_12"]
max_norm = 1
graph = create_graph(existing_odm, region, sites, max_norm)

images_folder = "/Users/jeandavidt/Downloads/plotly_images/"
graph.write_image(images_folder+"mtl_April1.png", width=1000, scale=3)
graph.write_image(images_folder+"mtl_April1.svg", width=1000)

no sars measurements found for site mtl_01
mtl_01
no sars measurements found for site mtl_05
mtl_05
no sars measurements found for site mtl_16
mtl_16
no sars measurements found for site mtl_12
mtl_12
